<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/deagwon/Dacon_satellite_%EC%97%B0%EC%9B%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%95%95%EC%B6%95%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
 
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
tf.__version__

'2.2.0-rc3'

## 데이터 압축해서 합치기

In [0]:
train = np.zeros([68706, 40, 40, 15], np.float32) # 전체 train_data shape, 자가 test용 데이터 제외
now_index = 0

for i in tqdm(range(9)):
    data = np.load(f'/content/drive/My Drive/2020 Kaggle Study/data/train/train{i}.npy')
    
    before_index = now_index
    now_index = data.shape[0] + now_index

    train[before_index:now_index,:,:,:] = data.astype(np.float32)
    del data
    gc.collect

test = np.load('/content/drive/My Drive/2020 Kaggle Study/data/train/test.npy').astype(np.float32)

print(train.shape, train.dtype)
print(test.shape, test.dtype)

100%|██████████| 9/9 [03:16<00:00, 21.83s/it]


(68706, 40, 40, 15) float32
(7639, 40, 40, 15) float32


In [0]:
np.save('/content/drive/My Drive/2020 Kaggle Study/data/32bits_data/train_32bits.npy', train)
np.save('/content/drive/My Drive/2020 Kaggle Study/data/32bits_data/test_32bits.npy', test)

## 자가 test set이랑 train set합치기

In [0]:
train = np.load('/content/drive/My Drive/2020 Kaggle Study/data/32bits_data/train_32bits.npy')
test = np.load('/content/drive/My Drive/2020 Kaggle Study/data/32bits_data/test_32bits.npy')

In [5]:
train.shape

(68706, 40, 40, 15)

In [6]:
test.shape

(7639, 40, 40, 15)

In [0]:
train = np.concatenate([train, test], axis = 0)

In [8]:
train.shape

(76345, 40, 40, 15)

In [0]:
np.save('/content/drive/My Drive/2020 Kaggle Study/data/train_total_32bits.npy', train)